**In this notebook, I try filling missing data (precinct) by two different ways:**

*   **Filling missing data (administrativeArea_1 column) by using geocoder library:**




In [ ]:
import pandas as pd
#read
dataset = pd.read_excel (r'/content/Test Data Analyst edited.xlsx')
#display
dataset.head()

,_id,formattedAddress,neighborhood,subLocality_2,administrativeArea_1,administrativeArea_2,administrativeArea_3,postalCode,streetNumber,route,country,accountStatus,interestsNormalized,additionalInfoProvided,gender,latitude,longitude,createdOn,ageGroupNormalized
0,5ee0bbb643d9f3001754b7d2,"4389 Nahdat Al Qalam, AR Rawdah District, Jedd...",AR Rawdah District,NaN,Makkah Province,NaN,NaN,23432.0,4389.0,Nahdat Al Qalam,Saudi Arabia,activated,"[""اللياقة والرياضة"",""الجمال والأزياء"",""الأم ال...",True,prefer_not_to_say,21.555095,39.165722,2020-06-10 16:53:42,18-25
1,5ee0d6b5cff9710017be41a9,2670 العميد، حي العزيزية، جدة 23334 8321، السع...,حي العزيزية,NaN,مكة المكرمة,NaN,NaN,23334.0,2670.0,العميد,السعودية,activated,"[""اللياقة والرياضة"",""الجمال والأزياء"",""الأم ال...",True,female,21.550321,39.180034,2020-06-10 18:48:53,36-45
2,5ee11b3839d22500172fe458,"8486 Al Maqsid Al Karim, Al Jami`ah, Jeddah 22...",Al Jami`ah,NaN,Makkah Province,NaN,NaN,22341.0,8486.0,Al Maqsid Al Karim,Saudi Arabia,activated,"[""الجمال والأزياء"",""المشروبات"",""العناية النسائ...",True,female,21.481793,39.246299,2020-06-10 23:41:12,18-25
3,5ee11b9939d22500172fe45c,"2823 Prince Saud Al Faisal, AR Rawdah District...",AR Rawdah District,NaN,Makkah Province,NaN,NaN,23431.0,2823.0,Prince Saud Al Faisal,Saudi Arabia,activated,"[""(الأكل للتسلية (شبس، حلا، مكسرات"",""العناية ب...",True,female,21.558208,39.150873,2020-06-10 23:42:49,36-45
4,5ee11c0f39d22500172fe463,6243 جبل جله، حي مشرفة، جدة 23331 2978، السعودية,حي مشرفة,NaN,مكة المكرمة,NaN,NaN,23331.0,6243.0,جبل جله,السعودية,activated,"[""الأم الرضع"",""مستلزمات الأطفال"",""(الأكل للتسل...",True,male,21.531778,39.183208,2020-06-10 23:44:47,26-35


In [ ]:
!pip install geopy

In [ ]:
# import module
from geopy.geocoders import Nominatim


In [ ]:
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")


In [ ]:
# find rows that are without province name
rows_without_province_name = dataset[dataset['administrativeArea_1'].isnull()]
print(rows_without_province_name)
#find address by latitude and longitude
for index, row in rows_without_province_name.iterrows():
  Latitude = str(row.latitude)
  Longitude = str(row.longitude)
  location = geolocator.reverse(Latitude+","+Longitude)
  if (index != 3315 and index != 3002): 
    #display the address 
    print(location)

                            _id  ... ageGroupNormalized
11     5ee11c5d39d22500172fe47d  ...              26-35
995    5ee5f0b6d20fd30017bd203c  ...              18-25
1205   5ee43a65d438c60017f6b197  ...              18-25
1427   5ee4b4b6d20fd30017bd1595  ...              18-25
3002   5ee2e984dacbcd00177790b0  ...              46-55
3315   5ee324a3dacbcd0017779671  ...              18-25
3758   5ee34e1bdacbcd0017779e19  ...              26-35
3920   5ee358c4dacbcd001777a0e9  ...              26-35
5301   5ee3c806d438c60017f6a926  ...              18-25
5764   5ee160a139d22500172ff665  ...              26-35
6617   5ee2012739d22500173005d3  ...              26-35
6647   5ee203f139d225001730065c  ...              36-45
7042   5ee20b5c39d2250017300cce  ...              18-25
7998   5ee2177439d2250017301d60  ...              36-45
8008   5ee217a839d2250017301d8e  ...              26-35
8296   5ee21c1139d22500173022a9  ...              18-25
10023  5ee24e1639d2250017304147  ...            

*   **Filling missing data (administrativeArea_1 column) by using ML (KNN algorithm):**

In [ ]:
#read training_dataset
training_dataset = pd.read_excel (r'/content/training dataset.xlsx')
#display
training_dataset.head()

,administrativeArea_1,latitude,longitude
0,Tabuk Province,28.358996,36.503661
1,Tabuk Province,28.408624,36.584514
2,Tabuk Province,28.431240,36.555689
3,Tabuk Province,28.452230,36.546338
4,Tabuk Province,28.417059,36.574782


In [ ]:
#read testing_dataset
testing_dataset = pd.read_excel (r'/content/testing dataset.xlsx')
#display
testing_dataset.head()

,administrativeArea_1,latitude,longitude
0,Al Baha Province,20.119129,41.404526
1,Al Baha Province,20.003410,41.469883
2,Al Baha Province,20.003691,41.469898
3,Al Baha Province,20.103132,41.288879
4,Al Baha Province,20.103434,41.289004


In [ ]:
#split datasets' columns to X (features) and Y (labels)
#training
X_train = training_dataset.iloc[:,1:3]
Y_train = training_dataset.iloc[:,0]
#testing
X_test = testing_dataset.iloc[:,1:3]
Y_test = testing_dataset.iloc[:,0]

In [ ]:
#import KNeighborsClassifier (KNN)
from sklearn.neighbors import KNeighborsClassifier as KNC
#model building for k=3
neigh=KNC(n_neighbors=3)
#train the model and test the model on training_dataset
neigh.fit(X_train,Y_train)
train_predict=neigh.predict(X_train)
pd.crosstab(train_predict,Y_train)

administrativeArea_1,Al Baha Province,Al Jouf Province,Al Madinah Province,Al Qassim,Aseer Province,Eastern Province,Hail Province,Jazan,Makkah Province,Najran Province,Northern Borders Province,Riyadh Province,Tabuk Province
row_0,,,,,,,,,,,,,
Al Baha Province,60,0,0,0,0,0,0,0,1,0,0,0,0
Al Jouf Province,0,19,0,0,0,0,0,0,0,0,0,0,0
Al Madinah Province,0,0,434,0,0,0,0,0,0,0,0,0,1
Al Qassim,0,0,0,266,0,0,0,0,0,0,0,0,0
Aseer Province,0,0,0,0,142,0,0,0,1,0,0,0,0
Eastern Province,0,0,0,0,0,738,0,0,0,0,0,0,0
Hail Province,0,0,0,0,0,0,33,0,0,0,0,0,0
Jazan,0,0,0,0,0,0,0,58,0,0,0,0,0
Makkah Province,0,0,0,0,1,0,0,0,3603,0,0,0,0


In [ ]:
#calculate training accuracy
train_acc=np.mean(train_predict==Y_train)
train_acc

0.9994471472799646

In [ ]:
#validate the model on testing_dataset
test_predict=neigh.predict(X_test)
pd.crosstab(test_predict,Y_test)

administrativeArea_1,Al Baha Province,Al Jouf Province,Al Madinah Province,Al Qassim,Aseer Province,Eastern Province,Hail Province,Jazan,Makkah Province,Najran Province,Northern Borders Province,Riyadh Province,Tabuk Province
row_0,,,,,,,,,,,,,
Al Baha Province,15,0,0,0,0,0,0,0,0,0,0,0,0
Al Jouf Province,0,4,0,0,0,0,0,0,0,0,0,0,0
Al Madinah Province,0,0,50,0,0,0,0,0,0,0,0,0,0
Al Qassim,0,0,0,25,0,0,0,0,0,0,0,0,0
Aseer Province,0,0,0,0,20,0,0,0,0,0,0,0,0
Eastern Province,0,0,0,0,0,100,0,0,0,0,2,0,0
Hail Province,0,0,0,0,0,0,10,0,0,0,0,0,0
Jazan,0,0,0,0,0,0,0,15,0,0,0,0,0
Makkah Province,0,0,0,0,0,0,0,0,1000,0,0,0,0


In [ ]:
#calculate testing accuracy
test_acc=np.mean(test_predict==Y_test)
test_acc

0.9991154356479434

In [ ]:
#to find best k value for building best model
acc=[]
for i in range(3,20,4):
    neigh=KNC(n_neighbors=i)
    neigh.fit(X_train,Y_train)
    train_acc=np.mean(neigh.predict(X_train)==Y_train)
    test_acc=np.mean(neigh.predict(X_test)==Y_test)
    acc.append([train_acc,test_acc])

acc

[[0.9994471472799646, 0.9991154356479434],
 [0.998562582927908, 0.9991154356479434],
 [0.9975674480318443, 0.9991154356479434],
 [0.9973463069438302, 0.9982308712958867],
 [0.9963511720477665, 0.9982308712958867]]

So, the best model when k= 3

**The end...**